@eidepozo

Fuentes: 

https://www.datacamp.com/community/tutorials/recommender-systems-python

https://www.kaggle.com/account/login?returnUrl=%2Frounakbanik%2Fthe-movies-dataset%2Fdata (dataset)

- **Recomendaciones simples:** ofrece recomendaciones generalizadas a cada usuario, basadas en la popularidad. La idea básica detrás de este sistema es que los items que son más populares tendrán una mayor probabilidad de ser del agrado de la audiencia 


- **Recomendaciones basadas en contenido:** sugieren elementos similares basados en un elemento en particular. La idea general detrás de estos sistemas de recomendación es que si a una persona le gustó un artículo en particular, también le gustará un artículo similar.


- **Motores de filtrado colaborativo:** estos sistemas intentan predecir la calificación o preferencia que un usuario otorgaría a un elemento en función de calificaciones y preferencias anteriores de otros usuarios. Los filtros de colaboración **no requieren metadatos de elementos** como sus contrapartes basadas en contenido.

## Simple recommender based in popularity/rating

<u>Resumen</u>

Este sistema utiliza el conteo general de votos TMDB y los promedios de votos para crear charts de películas principales


In [1]:
import pandas as pd

metadata = pd.read_csv('data/movies_metadata.csv', low_memory=False)

metadata.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


Se utiliza WR para que no se favorezcan las peliculas con menos votantes y buenas calificaciones

$$Weighted Rating (WR) = (\frac{v}{v+m}\cdot R)+(\frac{m}{v+m}\cdot C)$$

donde,

- $v$ es el número de votos por la pelicula `vote_count`
- $m$ es el minimo de votos requeridos para aparecer en la tabla
- $R$ es la calificación promedio de la pelicula `vote_average`
- $C$ es el promedio de los votos en todo el informe


In [2]:
#metadata.columns.tolist()

In [3]:
C = metadata['vote_average'].mean() #in a scale of 10
C

5.618207215133889

No existe un valor correcto para $m$, la selectividad depende del autor, en este caso se utiliza un percentil de 90 como punto de corte. En otras palabras, para que una película aparezca en las listas, debe tener más votos que al menos el 90% de las películas en la lista

In [4]:
# Calculate the minimum number of votes required to be in the chart, m
m = metadata['vote_count'].quantile(0.90)
print(m)

160.0


In [5]:
# Filter out all qualified movies into a new DataFrame
q_movies = metadata.copy().loc[metadata['vote_count'] >= m]
q_movies.shape

(4555, 24)

In [6]:
q_movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
5,False,NaN,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,1995-12-15,187436818.0,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0
8,False,NaN,35000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,9091,tt0114576,en,Sudden Death,International action superstar Jean Claude Van...,...,1995-12-22,64350171.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Terror goes into overtime.,Sudden Death,False,5.5,174.0


In [7]:
# Function that computes the weighted rating of each movie
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

In [8]:
# Define a new feature 'score' and calculate its value with `weighted_rating()`
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

In [9]:
# Sort movies based on score calculated above
q_movies = q_movies.sort_values('score', ascending=False)

# Print the top 15 movies
q_movies[['title', 'vote_count', 'vote_average', 'score']].head(15)

,title,vote_count,vote_average,score
314,The Shawshank Redemption,8358.0,8.5,8.445869
834,The Godfather,6024.0,8.5,8.425439
10309,Dilwale Dulhania Le Jayenge,661.0,9.1,8.421453
12481,The Dark Knight,12269.0,8.3,8.265477
2843,Fight Club,9678.0,8.3,8.256385
292,Pulp Fiction,8670.0,8.3,8.251406
522,Schindler's List,4436.0,8.3,8.206639
23673,Whiplash,4376.0,8.3,8.205404
5481,Spirited Away,3968.0,8.3,8.196055
2211,Life Is Beautiful,3643.0,8.3,8.187171


In [10]:
metadata2 = metadata.head(10000) # Personal technical limit

## Content based recommender

<u>Resumen</u> 
- Se crearon dos motores basados en contenido; uno considerando el resumen `overview` de la pelicula como entrada y el otro considerando metadatos como `cast`, `director`, `keywords` y `genres`.


- En el primer caso se calculan los vectores de **frecuencia de documento - frecuencia inversa de termino** (TF-IDF) para cada documento (frecuencia de una palabra que aparece en un documento, ponderada por la cantidad de documentos en los que aparece). Esto se hace para **reducir la importancia** de las palabras que aparecen con frecuencia en las vistas generales de la trama y, por lo tanto, su importancia para calcular el puntaje final de similitud.


- En el segundo caso se utiliza CountVectorizer() en lugar de TF-IDF. Esto es para **no minimizar** la presencia de un actor / director si el o ella ha actuado o dirigido en relativamente más peliculas.


- Posteriormente se calcula un puntaje de similitud (similarity score). Posibles candidatos mencionados: euclidiana, Pearson y **Coseno** (a usar)


- En el segundo caso es necesario convertir todos los string a lower case y eliminar los espacios (para diferenciar actores y personas con el mismo primer nombre por ej)


- En primer lugar, para esto, necesita una asignación inversa de títulos de películas e índices de DataFrame. En otras palabras, se requiere un mecanismo para identificar el índice de una película en su metadato DataFrame, dado su título.


- Finalmente para ambos casos el procedimiento es el mismo al definir `get_recommendations`: 

 - Obtener el índice de la película dado su título
 - Obtener la lista de puntajes de similitud de coseno para esa película en particular con todas las películas
 - Convertirlo en una lista de tuplas donde el primer elemento es su posición y el segundo es el puntaje de similitud
 - Ordenar la lista de tuplas mencionada anteriormente según los puntajes de similitud; es decir, el segundo elemento
 - Obtener los 10 elementos principales de esta lista
 - Ignorar el primer elemento, ya que se refiere a sí mismo (la película más similar a una película en particular es la película misma)
 - Retornar los títulos correspondientes a los índices de los elementos superiores
 

- Algunas sugerencias: incluir un filtro de popularidad, incluir otros miembros del cast como los productores, aumentar el peso del director, implementadas en: https://github.com/rounakbanik/movies/blob/master/movies_recommender.ipynb


- Desventajas de esta implementación en particular, **no es realmente personal** (a cualquier usuario le recomendaria lo mismo, no considera su perfil) y solo es capaz de sugerir peliculas cercanas (**no es capaz de capturar gustos** y recomendar otros generos)

In [11]:
# Print plot overviews of the first 5 movies.
metadata2['overview'].head()

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: overview, dtype: object

In [12]:
metadata2['overview'].isnull().sum()

29

In [13]:
# Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

# Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

# Replace NaN with an empty string
metadata2['overview'] = metadata2['overview'].fillna('')

# Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(metadata2['overview'])

# Output the shape of tfidf_matrix
tfidf_matrix.shape

/home/eidepozo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


(10000, 32350)

Esto nos dice que se usaron más de <u>32.000 palabras diferentes</u> para describir las 10000 películas en el conjunto de datos. Con esta matriz se puede calcular un puntaje de **similitud (similarity score)**. Existen varios candidatos como el puntaje de similitud **euclidiana**, de **Pearson** y **coseno**. Una vez más, no hay una respuesta correcta sobre qué puntaje es el mejor. Los diferentes puntajes funcionan bien en diferentes escenarios y, a menudo, es una buena idea experimentar con diferentes métricas.

Utilizará la similitud de coseno para calcular una cantidad numérica que denota la similitud entre dos películas. Utiliza el puntaje de similitud del coseno ya que es independiente de la magnitud y es relativamente fácil y rápido de calcular (especialmente cuando se usa junto con los puntajes TF-IDF, que se explicarán más adelante). Matemáticamente, se define de la siguiente manera:

$$cosine(x,y) = \frac{x\cdot y^T}{\|x\|\cdot\|y\|}$$

In [14]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

Definirá una función que toma el título de una película como entrada y genera una lista de las 10 películas más similares. En primer lugar, para esto, necesita una asignación inversa de títulos de películas e índices de DataFrame. En otras palabras, necesita un mecanismo para identificar el índice de una película en su metadato DataFrame, dado su título.

In [15]:
# Construct a reverse map of indices and movie titles
indices = pd.Series(metadata2.index, index=metadata2['title']).drop_duplicates()

In [16]:
indices.index 

Index(['Toy Story', 'Jumanji', 'Grumpier Old Men', 'Waiting to Exhale',
       'Father of the Bride Part II', 'Heat', 'Sabrina', 'Tom and Huck',
       'Sudden Death', 'GoldenEye',
       ...
       'Ike: Countdown to D-Day', 'Cry of the Banshee', 'Phantom Lady',
       'Act of Violence', 'Rouge', 'Miracle in Milan', 'Before the Fall',
       'The Frisco Kid', 'Onmyoji: The Yin Yang Master', 'State Property 2'],
      dtype='object', name='title', length=10000)

In [17]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices.get(key = title)

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return metadata2['title'].iloc[movie_indices]

In [18]:
get_recommendations('The Godfather')

1178     The Godfather: Part II
1914    The Godfather: Part III
8653               Violent City
6711                   Mobsters
6977            Queen of Hearts
8191                     Eulogy
2891             American Movie
4324                       Made
4464            Family Business
5689        The Young Americans
Name: title, dtype: object

La calidad del recomendador se puede mejorar usando mejor metadata. Se consideraran ahora: los 3 actores principales, el director, géneros relacionados y las palabras clave de la trama de la película. 

In [19]:
# Load keywords and credits
credits = pd.read_csv('data/credits.csv')
keywords = pd.read_csv('data/keywords.csv')

# Remove rows with bad IDs.
metadata = metadata.drop([19730, 29503, 35587])

# Convert IDs to int. Required for merging
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
metadata['id'] = metadata['id'].astype('int')

# Merge keywords and credits into your main metadata dataframe
metadata = metadata.merge(credits, on='id')
metadata = metadata.merge(keywords, on='id')

In [20]:
# Print the first two movies of your newly merged metadata
metadata.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."


In [21]:
# Parse the stringified features into their corresponding python objects
from ast import literal_eval

features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
    metadata[feature] = metadata[feature].apply(literal_eval)

In [22]:
# Import Numpy 
import numpy as np

In [23]:
# Get the director's name from the crew feature. If director is not listed, return NaN
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [24]:
# Returns the list top 3 elements or entire list; whichever is more.
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []

In [25]:
# Define new director, cast, genres and keywords features that are in a suitable form.
metadata['director'] = metadata['crew'].apply(get_director)

features = ['cast', 'keywords', 'genres']
for feature in features:
    metadata[feature] = metadata[feature].apply(get_list)

In [26]:
# Print the new features of the first 3 films
metadata[['title', 'cast', 'director', 'keywords', 'genres']].head(3)

,title,cast,director,keywords,genres
0,Toy Story,"[Tom Hanks, Tim Allen, Don Rickles]",John Lasseter,"[jealousy, toy, boy]","[Animation, Comedy, Family]"
1,Jumanji,"[Robin Williams, Jonathan Hyde, Kirsten Dunst]",Joe Johnston,"[board game, disappearance, based on children'...","[Adventure, Fantasy, Family]"
2,Grumpier Old Men,"[Walter Matthau, Jack Lemmon, Ann-Margret]",Howard Deutch,"[fishing, best friend, duringcreditsstinger]","[Romance, Comedy]"


El siguiente paso se realiza para que el vectorizador no cuente a "Johnny" de "Johnny Deep" y "Johnny Galecki" como lo mismo, pasan a ser `johnnydeep` y `johnyygalecki`

In [27]:
# Function to convert all strings to lower case and strip names of spaces
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [28]:
# Apply clean_data function to your features.
features = ['cast', 'keywords', 'director', 'genres']

for feature in features:
    metadata[feature] = metadata[feature].apply(clean_data)

In [29]:
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])

In [30]:
# Create a new soup feature
metadata['soup'] = metadata.apply(create_soup, axis=1)

In [31]:
metadata3 = metadata.head(10000)

Los siguientes pasos son los mismos que hiciste con tu recomendación basada en la descripción de la trama. Una diferencia importante es que usa CountVectorizer () en lugar de TF-IDF. Esto se debe a que no desea minimizar la presencia de un actor / director si él o ella ha actuado o dirigido en relativamente más películas. No tiene mucho sentido intuitivo

In [32]:
# Import CountVectorizer and create the count matrix
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(metadata3['soup'])

In [33]:
# Compute the Cosine Similarity matrix based on the count_matrix
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [34]:
# Reset index of your main DataFrame and construct reverse mapping as before
metadata3 = metadata3.reset_index()
indices = pd.Series(metadata3.index, index=metadata3['title'])

In [35]:
get_recommendations('The Godfather', cosine_sim2)

1934    Darby O'Gill and the Little People
1199                             Manhattan
8001                    The Creeping Flesh
7772                     This Island Earth
1186                            Annie Hall
1648                                Kundun
3487                      Regret to Inform
4012                        My Demon Lover
5309                          Juwanna Mann
5                                     Heat
Name: title, dtype: object

## Collaborative Filtering

<u>Resumen</u>

- Utiliza SVD (Singular Value Decomposition) para minimizar el error medio cuadratico (RMSE)



Nuestro motor basado en contenido tiene algunas limitaciones severas. Solo es capaz de sugerir películas cercanas a una película determinada. Es decir, no es capaz de capturar gustos y proporcionar recomendaciones en todos los géneros.

Además, el motor que creamos no es realmente personal, ya que no captura los gustos y prejuicios personales de un usuario. Cualquier persona que consulte nuestro motor para obtener recomendaciones basadas en una película recibirá las mismas recomendaciones para esa película, independientemente de quién sea.

Por lo tanto, en esta sección, utilizaremos una técnica llamada Filtrado colaborativo para hacer recomendaciones a los Observadores de películas. El Filtrado Colaborativo se basa en la idea de que los usuarios similares a mí pueden usarse para predecir cuánto me gustará un producto o servicio en particular que esos usuarios hayan usado / experimentado, pero yo no.

No implementaré el filtrado colaborativo desde cero. En cambio, usaré la biblioteca Surprise que utilizó algoritmos extremadamente potentes como la Descomposición de valor singular (SVD) para minimizar RMSE (Root Mean Square Error) y dar excelentes recomendaciones.

In [36]:
from surprise import Reader, Dataset, SVD

In [37]:
reader = Reader()

In [38]:
ratings = pd.read_csv('data/ratings_small.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


https://surprise.readthedocs.io/en/stable/getting_started.html#automatic-cross-validation

In [39]:
from surprise.model_selection import cross_validate

In [40]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

svd = SVD()

# Run 5-fold cross-validation and print results
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9016  0.8926  0.8932  0.8992  0.8971  0.8967  0.0035  
MAE (testset)     0.6935  0.6891  0.6868  0.6925  0.6899  0.6904  0.0024  
Fit time          4.33    4.16    4.14    4.14    4.15    4.18    0.07    
Test time         0.69    5.70    0.13    0.13    0.13    1.36    2.18    


{'test_rmse': array([0.90160835, 0.89256766, 0.89317397, 0.89919548, 0.89712228]),
 'test_mae': array([0.69352261, 0.68910982, 0.68681331, 0.69254013, 0.68985107]),
 'fit_time': (4.332402944564819,
  4.159217357635498,
  4.141799211502075,
  4.1380839347839355,
  4.151382923126221),
 'test_time': (0.6854453086853027,
  5.700213193893433,
  0.13132977485656738,
  0.13186144828796387,
  0.13377928733825684)}

Obtenemos un error cuadrático medio de 0.8974 que es más que suficiente para nuestro caso. Ahora entrenemos en nuestro conjunto de datos y lleguemos a las predicciones.

In [41]:
# En el caso de no querer correr un procedimiento de full cross-validation 
from surprise.model_selection import train_test_split

# test set is made of 25% of the ratings.
trainset, testset = train_test_split(data, test_size=.25)

#svd = SVD() sera necesario definirlo denuevo?
# Train the algorithm on the trainset
svd.fit(trainset)

In [42]:
ratings[ratings['userId'] == 1]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [43]:
from surprise import accuracy

# Predict ratings for the testset
predictions = svd.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 0.9072


0.9071806534655509

In [44]:
svd.predict(1, 302, 3)

Prediction(uid=1, iid=302, r_ui=3, est=2.7438354674461545, details={'was_impossible': False})

Para la pelicula con ID 302, obtenemos una predicción estimada de 2.81. Una característica sorprendente de este sistema de recomendación es que no le importa qué película es (o qué contiene). Funciona únicamente sobre la base de una ID de película asignada e intenta predecir las clasificaciones en función de cómo los otros usuarios han predicho la película.